In [5]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news2_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

In [6]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [4]:
docID_assign_z = {}
m_z = {}
n_z = {}
n_w = {}
n_b = {}
Topics = []
V = set()
D = set()
alpha = 0.6
beta = 0.02
total_iter = 10
for iter in range(total_iter):
    for docID in range(len(corpus)):
        if docID % 1000 == 0:
            print("iter: ",iter," ,docID ",docID)
            print("len(Topics) ",len(Topics))
        words = corpus[docID]
        D.discard(docID)
        if docID in docID_assign_z.keys():
            before_k = docID_assign_z[docID]
            m_z[before_k].discard(docID)
            for biterm in words:
                for word in biterm.split(','):
                    n_z[before_k][word] -= 1
                    n_w[before_k] -=1
        else:
            before_k = -1
        if len(D) == 0 and len(V) == 0:
            choose_k = 0
            D.add(docID)
            docID_assign_z[docID] = choose_k
            if choose_k not in m_z.keys():
                m_z[choose_k] = set()
            m_z[choose_k].add(docID)
            for biterm in words:
                for word in biterm.split(','):
                    if choose_k not in n_w.keys():
                        n_w[choose_k] = 0
                    if choose_k not in n_z.keys():
                        n_z[choose_k] = {}
                    if word not in n_z[choose_k].keys():
                        n_z[choose_k][word] = 0
                    n_z[choose_k][word] += 1
                    n_w[choose_k] += 1
                    V.add(word)
            if choose_k not in Topics:
                Topics.append(choose_k)
        else:
            log_pro = []
            for k in Topics:
                pro_k = len(m_z[k])
                if pro_k != 0:
                    i = 0
                    for biterm in words:
                        each_word = biterm.strip().split(',')
                        for word in each_word:
                            if word not in n_z[k].keys():
                                n_z[k][word] = 0
                        for j in range(doc_biterm[docID][biterm]):
                            pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                            i += 1

                if pro_k == 0:
                    pro_k = sys.float_info.min
                log_pro.append(pro_k)

            pro_new_k = alpha*(len(D))
            i = 0
            for biterm in words:
                for j in range(doc_biterm[docID][biterm]):
                    pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                    i += 1

            if pro_new_k == 0:
                pro_new_k = sys.float_info.min        
            log_pro.append(pro_new_k)

            sum_pro=sum(log_pro)

            normalized_posterior = [i/sum_pro for i in log_pro]    
            select_k = None
            if iter == (total_iter - 1):
                select_k = normalized_posterior.index(max(normalized_posterior))

            else:
                select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

            if select_k == len(Topics):
                choose_k = np.max(Topics) + 1
            else:
                choose_k = Topics[select_k]

            D.add(docID)
            docID_assign_z[docID] = choose_k
            if choose_k not in m_z.keys():
                m_z[choose_k] = set()
            m_z[choose_k].add(docID)
            for biterm in words:
                for word in biterm.split(','):
                    if choose_k not in n_w.keys():
                        n_w[choose_k] = 0
                    if choose_k not in n_z.keys():
                        n_z[choose_k] = {}
                    if word not in n_z[choose_k].keys():
                        n_z[choose_k][word] = 0
                    n_z[choose_k][word] += 1
                    n_w[choose_k] += 1
                    V.add(word)
            if choose_k not in Topics:
                Topics.append(choose_k)

        count_k = []
        for k in Topics:
            if k in m_z.keys() and len(m_z[k]) == 0:
                m_z.pop(k, None)
                n_z.pop(k, None)
                n_w.pop(k, None)
                count_k.append(k)
        for k in count_k:
            Topics.remove(k)   

    from sklearn.metrics.cluster import normalized_mutual_info_score
    nmi_sample = []
    nmi_result = []
    for key, value in news_labels.items():
            nmi_sample.append(value)
            nmi_result.append(docID_assign_z[key])
    print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
    alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  119


KeyboardInterrupt: 

In [28]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.8
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
         start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  694 topics:  120 truth:  132  NMI:  0.8882984363075321
start  0  end  1388 topics:  131 truth:  143  NMI:  0.8655685741007803
start  695  end  2082 topics:  134 truth:  146  NMI:  0.8601966096751001
start  1389  end  2776 topics:  125 truth:  142  NMI:  0.8653117694146474
start  2083  end  3470 topics:  131 truth:  143  NMI:  0.8691099008525507
start  2777  end  4164 topics:  130 truth:  140  NMI:  0.870470086290841
start  3471  end  4858 topics:  131 truth:  142  NMI:  0.8850242362688984
start  4165  end  5552 topics:  126 truth:  137  NMI:  0.8614462503512703
start  4859  end  6246 topics:  134 truth:  139  NMI:  0.8526164464109245
start  5553  end  6940 topics:  137 truth:  137  NMI:  0.8724485266855552
start  6247  end  7634 topics:  130 truth:  139  NMI:  0.8775498841669781
start  6941  end  8328 topics:  131 truth:  144  NMI:  0.8580625490011807
start  7635  end  9022 topics:  141 truth:  147  NMI:  0.8604648478317101
start  8329  end  9716 topics:  132 truth:  147

In [22]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8791632071654407, 0.8689766289541706, 0.8520647369847433, 0.8672141455434031, 0.8669673186133002, 0.8625201831434595, 0.8767862446229376, 0.8581853527328299, 0.8442325374746548, 0.8599915553662428, 0.8743825671633735, 0.8612640060536784, 0.8519235493142886, 0.8691149373848646, 0.86462425414101, 0.8621060128272594]

0.8637198273428535


In [18]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print()

print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8730133849433831, 0.8684743694996326, 0.8804032259273199, 0.8758633536436381, 0.8656884137743128, 0.8843568887366188, 0.8758842887943914, 0.8596648235205762, 0.8664921845605948, 0.8776708653357372, 0.875552963171944, 0.8499807697201824, 0.8813713693485388, 0.871907891615146, 0.8641084076115443, 0.8646719585908315]

0.8709440724246496


In [17]:
print(np.mean(item_NMI))
print(beta)

0.8709440724246496
0.02


In [30]:
print(NMI_batch)
print(beta)
print(NMI_batch.keys())
print(NMI_batch.values())

{1: 0.8882984363075321, 2: 0.8655685741007803, 3: 0.8601966096751001, 4: 0.8653117694146474, 5: 0.8691099008525507, 6: 0.870470086290841, 7: 0.8850242362688984, 8: 0.8614462503512703, 9: 0.8526164464109245, 10: 0.8724485266855552, 11: 0.8775498841669781, 12: 0.8580625490011807, 13: 0.8604648478317101, 14: 0.8698139510080556, 15: 0.8714141829613143, 16: 0.8574559445509717}
0.02
dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([0.8882984363075321, 0.8655685741007803, 0.8601966096751001, 0.8653117694146474, 0.8691099008525507, 0.870470086290841, 0.8850242362688984, 0.8614462503512703, 0.8526164464109245, 0.8724485266855552, 0.8775498841669781, 0.8580625490011807, 0.8604648478317101, 0.8698139510080556, 0.8714141829613143, 0.8574559445509717])


In [35]:
nmi_batch = {}
topic_batch = {}
start = 0
end = 0
total_batch = None
if len(corpus) % 500 == 0:
    total_batch = int(len(corpus) / 500)
else:
    total_batch = int(len(corpus) / 500) +1

for batch in range(total_batch):
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.002
    end = 500 * (batch + 1)
    if end > len(corpus):
        end = len(corpus)
    nmi_batch[batch] = 0
    topic_batch[batch] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            nmi_batch[batch] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    start = end 
 
print(nmi_batch.keys())
print(nmi_batch.values())

start  0  end  500 topics:  121 truth:  119  NMI:  0.8816253485292893
start  500  end  1000 topics:  120 truth:  118  NMI:  0.8896958498673855
start  1000  end  1500 topics:  124 truth:  120  NMI:  0.887547430347356
start  1500  end  2000 topics:  114 truth:  120  NMI:  0.8798078001390809
start  2000  end  2500 topics:  115 truth:  117  NMI:  0.8758785113006561
start  2500  end  3000 topics:  129 truth:  125  NMI:  0.8878789420209223
start  3000  end  3500 topics:  121 truth:  119  NMI:  0.8858948989230734
start  3500  end  4000 topics:  126 truth:  120  NMI:  0.8956560454451744
start  4000  end  4500 topics:  118 truth:  120  NMI:  0.8906988343883995
start  4500  end  5000 topics:  118 truth:  118  NMI:  0.8759524080214447
start  5000  end  5500 topics:  114 truth:  106  NMI:  0.8669809275843139
start  5500  end  6000 topics:  121 truth:  113  NMI:  0.8724444083741711
start  6000  end  6500 topics:  116 truth:  107  NMI:  0.8799065794215711
start  6500  end  7000 topics:  123 truth:  

In [47]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.006
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  694 topics:  138 truth:  132  NMI:  0.8909870709330914
start  0  end  1388 topics:  155 truth:  143  NMI:  0.8651699783582908
start  695  end  2082 topics:  164 truth:  146  NMI:  0.8672140012352513
start  1389  end  2776 topics:  142 truth:  142  NMI:  0.8680859675421644
start  2083  end  3470 topics:  153 truth:  143  NMI:  0.8717006857362761
start  2777  end  4164 topics:  153 truth:  140  NMI:  0.8744622753269023
start  3471  end  4858 topics:  154 truth:  142  NMI:  0.8800267550934175
start  4165  end  5552 topics:  153 truth:  137  NMI:  0.8678603290385756
start  4859  end  6246 topics:  150 truth:  139  NMI:  0.8516026112257538
start  5553  end  6940 topics:  156 truth:  137  NMI:  0.8706066920056388
start  6247  end  7634 topics:  147 truth:  139  NMI:  0.8771950844789724
start  6941  end  8328 topics:  161 truth:  144  NMI:  0.8670647824143706
start  7635  end  9022 topics:  167 truth:  147  NMI:  0.8688979969642097
start  8329  end  9716 topics:  151 truth:  14

In [48]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8909870709330914, 0.8651699783582908, 0.8672140012352513, 0.8680859675421644, 0.8717006857362761, 0.8744622753269023, 0.8800267550934175, 0.8678603290385756, 0.8516026112257538, 0.8706066920056388, 0.8771950844789724, 0.8670647824143706, 0.8688979969642097, 0.8707276568667933, 0.8661519878300761, 0.8629784090033633]
0.8700457677533218


In [7]:
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    

start  0  end  694 topics:  113 truth:  132  NMI:  0.8836736821351552
start  0  end  1388 topics:  127 truth:  143  NMI:  0.8678265020507938
start  695  end  2082 topics:  134 truth:  146  NMI:  0.864495267593281
start  1389  end  2776 topics:  123 truth:  142  NMI:  0.8706820058256952
start  2083  end  3470 topics:  127 truth:  143  NMI:  0.8631459468372141
start  2777  end  4164 topics:  124 truth:  140  NMI:  0.8708992967204708
start  3471  end  4858 topics:  123 truth:  142  NMI:  0.8765553290209164
start  4165  end  5552 topics:  121 truth:  137  NMI:  0.8551546457480605
start  4859  end  6246 topics:  126 truth:  139  NMI:  0.846603071665342
start  5553  end  6940 topics:  131 truth:  137  NMI:  0.8639294618581334
start  6247  end  7634 topics:  122 truth:  139  NMI:  0.8799141431225486
start  6941  end  8328 topics:  134 truth:  144  NMI:  0.8568565127760674
start  7635  end  9022 topics:  132 truth:  147  NMI:  0.8540550286342588
start  8329  end  9716 topics:  121 truth:  147 

In [8]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8836736821351552, 0.8678265020507938, 0.864495267593281, 0.8706820058256952, 0.8631459468372141, 0.8708992967204708, 0.8765553290209164, 0.8551546457480605, 0.846603071665342, 0.8639294618581334, 0.8799141431225486, 0.8568565127760674, 0.8540550286342588, 0.8645514921543302, 0.8606425327935067, 0.8625456833956983]
0.8650956626457169


In [9]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.6
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
    if batch_i != 1:
        start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  694 topics:  112 truth:  132  NMI:  0.8762459039342183
[27.288816929872613]
start  0  end  1388 topics:  128 truth:  143  NMI:  0.860589449245971
[27.288816929872613, 59.55673653274858]
start  695  end  2082 topics:  127 truth:  146  NMI:  0.8657264330184065
[27.288816929872613, 59.55673653274858, 64.34812713088947]
start  1389  end  2776 topics:  121 truth:  142  NMI:  0.860015590313485
[27.288816929872613, 59.55673653274858, 64.34812713088947, 60.149568120706846]
start  2083  end  3470 topics:  128 truth:  143  NMI:  0.8711558438895886
[27.288816929872613, 59.55673653274858, 64.34812713088947, 60.149568120706846, 58.97903506745081]
start  2777  end  4164 topics:  127 truth:  140  NMI:  0.8728882689844335
[27.288816929872613, 59.55673653274858, 64.34812713088947, 60.149568120706846, 58.97903506745081, 56.51260873601956]
start  3471  end  4858 topics:  126 truth:  142  NMI:  0.8746967476654431
[27.288816929872613, 59.55673653274858, 64.34812713088947, 60.149568120706846,

In [ ]:
0.8700457677533218

In [39]:
common_keys = []

item_NMI = []
for key,value in sorted(NMI_batch.items(), key=lambda x: x[0]):
    common_keys.append(key)
    item_NMI.append(value)
print(common_keys)

print()

print(item_NMI)
print(np.mean(item_NMI))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0.8803076523452149, 0.8687252927079312, 0.8690938337150865, 0.8724089857949292, 0.8682097627878833, 0.8829909798277922, 0.884561511160292, 0.8632691421539366, 0.8562621211408585, 0.8700285470763935, 0.8795298234923449, 0.8683806692576711, 0.8619609115942072, 0.8747436197854798, 0.8670512491432407, 0.856084078731455]
0.8702255112946697


In [10]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    
    total_batch = 16
    NMI_batch = {}
    start = 0
    end = 0
    topic_batch = {}
    time_iteration_list = []
    for batch_i in range(1,17):
        if batch_i == 16:
            end = int(len(corpus))
        else:
            end = int(len(corpus) / total_batch) * batch_i

        docID_assign_z = {}
        m_z = {}
        n_z = {}
        n_w = {}
        n_b = {}
        Topics = []
        V = set()
        D = set()
        #alpha = 0.6
        beta = 0.02
        NMI_batch[batch_i] = 0
        topic_batch[batch_i] = 0

        time_start = timeit.default_timer()
        total_iter = 10
        for iter in range(total_iter):
            for docID in range(start,end):
                words = corpus[docID]
                D.discard(docID)
                if docID in docID_assign_z.keys():
                    before_k = docID_assign_z[docID]
                    m_z[before_k].discard(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            n_z[before_k][word] -= 1
                            n_w[before_k] -=1
                else:
                    before_k = -1
                if len(D) == 0 and len(V) == 0:
                    choose_k = 0
                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)
                else:
                    log_pro = []
                    for k in Topics:
                        pro_k = len(m_z[k])
                        if pro_k != 0:
                            i = 0
                            for biterm in words:
                                each_word = biterm.strip().split(',')
                                for word in each_word:
                                    if word not in n_z[k].keys():
                                        n_z[k][word] = 0
                                for j in range(doc_biterm[docID][biterm]):
                                    pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                    i += 1

                        if pro_k == 0:
                            pro_k = sys.float_info.min
                        log_pro.append(pro_k)

                    pro_new_k = alpha*(len(D))
                    i = 0
                    for biterm in words:
                        for j in range(doc_biterm[docID][biterm]):
                            pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                            i += 1

                    if pro_new_k == 0:
                        pro_new_k = sys.float_info.min        
                    log_pro.append(pro_new_k)

                    sum_pro=sum(log_pro)

                    normalized_posterior = [i/sum_pro for i in log_pro]    
                    select_k = None
                    if iter == (total_iter - 1):
                        select_k = normalized_posterior.index(max(normalized_posterior))

                    else:
                        select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                    if select_k == len(Topics):
                        choose_k = np.max(Topics) + 1
                    else:
                        choose_k = Topics[select_k]

                    D.add(docID)
                    docID_assign_z[docID] = choose_k
                    if choose_k not in m_z.keys():
                        m_z[choose_k] = set()
                    m_z[choose_k].add(docID)
                    for biterm in words:
                        for word in biterm.split(','):
                            if choose_k not in n_w.keys():
                                n_w[choose_k] = 0
                            if choose_k not in n_z.keys():
                                n_z[choose_k] = {}
                            if word not in n_z[choose_k].keys():
                                n_z[choose_k][word] = 0
                            n_z[choose_k][word] += 1
                            n_w[choose_k] += 1
                            V.add(word)
                    if choose_k not in Topics:
                        Topics.append(choose_k)

                count_k = []
                for k in Topics:
                    if k in m_z.keys() and len(m_z[k]) == 0:
                        m_z.pop(k, None)
                        n_z.pop(k, None)
                        n_w.pop(k, None)
                        count_k.append(k)
                for k in count_k:
                    Topics.remove(k)   
            if iter == 9:
                from sklearn.metrics.cluster import normalized_mutual_info_score
                nmi_sample = []
                nmi_result = []
                for key, value in news_labels.items():
                    if key < end and key >= start:
                        nmi_sample.append(value)
                        nmi_result.append(docID_assign_z[key])
                NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
                topic_batch[batch_i] = len(Topics)
                #print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        if batch_i != 1:
            start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    
    alpha_NMI[alpha] = np.mean(list(NMI_batch.values()))
    alpha_topics[alpha] = int(np.mean(list(topic_batch.values())))
    print("alpha ",alpha, " NMI: ", alpha_NMI[alpha])
    print("alpha ",alpha, " Topics: ", alpha_topics[alpha])

alpha  0.1  NMI:  0.8553973896584917
alpha  0.1  Topics:  109
alpha  0.2  NMI:  0.8586395598047273
alpha  0.2  Topics:  113
alpha  0.3  NMI:  0.8621052092575683
alpha  0.3  Topics:  117
alpha  0.4  NMI:  0.8617021553300233
alpha  0.4  Topics:  121
alpha  0.5  NMI:  0.8619820784046283
alpha  0.5  Topics:  124
alpha  0.6  NMI:  0.8652709179319698
alpha  0.6  Topics:  126
alpha  0.7  NMI:  0.865502768912514
alpha  0.7  Topics:  127
alpha  0.8  NMI:  0.8665753557337122
alpha  0.8  Topics:  129
alpha  0.9  NMI:  0.8666900836555417
alpha  0.9  Topics:  132
alpha  1.0  NMI:  0.8679872662269225
alpha  1.0  Topics:  132


In [11]:
print(alpha_NMI.values())
print(alpha_topics.values())

dict_values([0.8553973896584917, 0.8586395598047273, 0.8621052092575683, 0.8617021553300233, 0.8619820784046283, 0.8652709179319698, 0.865502768912514, 0.8665753557337122, 0.8666900836555417, 0.8679872662269225])
dict_values([109, 113, 117, 121, 124, 126, 127, 129, 132, 132])
